# Topic Modeling for Everybody with Google Colab

**Super simple topic modeling using both the Non Negative Matrix Factorization (NMF) and Latent Dirichlet Allocation (LDA) algorithms.**

This Google Colab Notebook makes topic modeling accessible to everybody. Textual data can be loaded from a Google Sheet and topics derived from NMF and LDA can be generated. Only simple form entry is required to set:

* the name of the google sheet
* the number of topics to be generated
* the number of top words and documents that must be printed out for each topic





In [ ]:
#@title Install gspread, authenticate and load data from a Google Sheet
!pip install --upgrade -q gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

# Default data from
# http://web.eecs.utk.edu/~berry/order/node4.html#SECTION00022000000000000000

googlesheet_filename = 'tdc' #@param {type:"string"}
data_rows_to_preview = 15 #@param {type:"integer"}


In [ ]:
#@title Load and preview data from a Google Sheet

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open(googlesheet_filename).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# convert the 2nd column values to a list
documents = []
for row in rows[1:]:
  documents.append(row[2])
  
#print(documents)

# Convert to a DataFrame and render.
import pandas as pd
dataset_df = pd.DataFrame.from_records(rows)
dataset_df.head(n=data_rows_to_preview)


,0,1,2
0,,label,text
1,0,__label__Du_lich,hành_trình tour du_lịch mỹ bờ đông du_khách lữ...
2,1,__label__Nha_dat,phòng quanh khu_vực hồ tùng mậu phòng khép_kín...
3,2,__label__Nha_dat,thuê nhà_riêng dt sàn phòng ngủ thoáng mát an_...
4,3,__label__Nha_dat,thuê tầng khép_kín nguyễn_khoái bếp đầy_đủ tủ_...
5,4,__label__Mua_sam,crumpler jackpack full photo giá vnđ vnđ đựng ...
6,5,__label__Nha_dat,đăng hộ chủ ngõ đào_tấn lotte công_viên thủ_lệ...
7,6,__label__Tai_chinh,công_ty phát_sinh công_việc bắt_buộc nhân_viên...
8,7,__label__Mua_sam,pin hãng samsung galaxy_grand prime g cell mad...
9,8,__label__Du_lich,miền đất mạc dubai mênh_mông sa_mạc dubai nhỏ_...




---



---



In [ ]:
#@title Set topic modeling algorithm arguments

no_topics =  23#@param {type:"integer"}

no_top_words =  5#@param {type:"integer"}

no_top_documents =  20#@param {type:"integer"}

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np

In [ ]:
#@title Run NMF

def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# Run NMF
nmf_model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
nmf_W = nmf_model.transform(tfidf)
nmf_H = nmf_model.components_

print("NMF Topics")
display_topics(nmf_H, nmf_W, tfidf_feature_names, documents, no_top_words, no_top_documents)
print("--------------")



NMF Topics
Topic 0:
phòng khép_kín chủ xe giá
phòng trọ chủ khép_kín full đồ phòng tầm ngõ trần_duy_hưng khu trung_tâm big c chủ giờ_giấc thoải_mái phòng khép_kín nóng lạnh điều_hoà giường tủ lắp máy_giặt điện giá hoá_đơn nhà_nước gửi xe tầng free camera giá r liên_hệ phòng thoải_mái
phòng trọ r đổ quanh khu_vực hoàng quốc việt phạm văn đồng hồ tùng mậu phòng vệ_sinh khép_kín chỗ nấu_ăn chủ giờ_giấc thoải_mái đóng tiền nóng lạnh phòng thuê nhượng phòng ko
t phòng quanh khu xuân_thủy cầu giấy mỹ_đình phòng khép_kín nóng lạnh giờ_giấc thoải_mái giá phòng triệu nhượng phòng giới_thiệu t t
chủ thuê phòng ngõ trần_duy_hưng phòng thoáng mát sạch_sẽ nóng lạnh cơ_sở vật_chất camera bảo_vệ an_ninh vệ_sinh diện_tích chỗ xe free chủ giờ_giấc tự_do phòng to r tầng phí dịch_vụ tiền điện k tiền internet phòng vệ_sinh phòng liên_hệ
e chào m n e giúp_đỡ m n chút e phòng trọ nữ quanh nguyễn_trãi bán_kính m khu_vực phùng_khoang triều_khúc phòng giá r đổ k phòng sạch_sẽ thoáng mát k hòa nóng lạnh giường 

In [ ]:
#@title Run LDA

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

# Run LDA
lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

print("LDA Topics")
display_topics(lda_H, lda_W, tf_feature_names, documents, no_top_words, no_top_documents)

LDA Topics
Topic 0:
thuế kế_toán công_ty nộp lương
kế_toán an_phát kỹ_năng kiểm_tra sổ_sách kế_toán bctc nhật_ký rà_soát định khoản kế_toán định khoản đối_ứng nợ kiểm_tra tiền kết tổng_phát_sinh nhật_ký tổng_phát_sinh bảng cân_đối tài_khoản bảng cân_đối tài_khoản tổng_số_dư_nợ đầu kỳ tổng_số dư đầu kỳ số_dư kỳ kết tổng_phát_sinh nợ kỳ tổng_phát_sinh kỳ tổng_phát_sinh nhật_ký kỳ tổng_số_dư nợ kỳ tổng_số_dư kỳ nguyên_tắc tổng_phát_sinh nợ tổng_phát_sinh tài_khoản ko số_dư trừ lưỡng dư nợ ko số_dư kỳ âm tải khoản ko dư nợ trừ lưỡng số_dư ko số_dư kỳ âm tài_khoản sổ cân_đối phát_sinh > sai kết tài_khoản tiền_mặt số_dư_nợ đầu kỳ sổ tk số_dư_nợ đầu kỳ tk bảng cân_đối phát_sinh số_dư nợ đầu kỳ sổ quỹ tiền_mặt phát_sinh nợ sổ tk phát_sinh nợ tk bảng cân_đối phát_sinh phát_sinh nợ sổ quỹ tiền_mặt số_dư_nợ kỳ sổ tk số_dư kỳ tk bảng cân_đối phát_sinh số_dư nợ kỳ sổ quỹ tiền_mặt tài_khoản tiền gửi ngân_hàng số_dư_nợ đầu kỳ sổ tk số_dư_nợ đầu kỳ tk bảng cân_đối phát_sinh số_dư nợ đầu kỳ sổ tiền gửi